In [2]:
from neuralprophet import NeuralProphet

import pandas as pd
pd.set_option('precision', 2)
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import optuna
import data
import os
import os.path
import importlib
importlib.reload(data)

from data import update_expl_data, update_target_data, food_categories, preprocess_expl

In [3]:
"""
Set sample rate. In this notebook, all data will be resampled at the chosen frequency.
'MS' : Monthly (Month Start)
'W' : Weekly
'D' : Daily
"""

year_period = {'MS': 12, 'W': 52, 'D': 365}
frequency = 'MS'
one_year = year_period[frequency]

## Load Data Using APIs

In [4]:
"""
Load food CPI data from January 1986 to the most recently available data.
"""

foodprice_df = update_target_data(food_categories, './data_files/food_cpi.csv')
foodprice_df = foodprice_df.resample(frequency).mean().interpolate()
foodprice_df

,Bakery and cereal products (excluding baby food),Dairy products and eggs,"Fish, seafood and other marine products",Food purchased from restaurants,Food,"Fruit, fruit preparations and nuts",Meat,Other food products and non-alcoholic beverages,Vegetables and vegetable preparations
REF_DATE,,,,,,,,,
1986-01-01,69.3,70.9,60.6,59.1,67.3,76.0,65.1,77.5,76.0
1986-02-01,70.3,70.8,61.3,59.1,66.9,77.6,64.2,78.1,68.4
1986-03-01,70.6,71.1,61.3,59.3,67.0,79.2,64.2,78.6,66.2
1986-04-01,71.3,71.0,61.4,59.7,67.7,82.2,63.6,79.5,71.1
1986-05-01,71.2,71.4,61.9,59.9,68.2,83.5,64.0,79.8,75.3
...,...,...,...,...,...,...,...,...,...
2021-05-01,157.8,146.6,147.6,163.5,156.6,143.9,175.4,141.6,153.8
2021-06-01,157.7,145.3,146.2,163.9,156.8,144.5,176.7,142.2,153.4
2021-07-01,157.9,146.4,146.6,165.2,157.6,141.7,180.9,141.9,154.8


In [5]:
"""
Load exogenous/auxiliary explanatory variables from FRED: https://fred.stlouisfed.org/
These data sources reflect various economic factors that may improve forecasts. 
Please visit the FRED website to learn more about these series, and to find others
that may be useful for food CPI forecasting. 
"""

data_sources = ["DEXCAUS",
                "DCOILWTICO",
                "WILL5000IND",
                "VXOCLS",
                "CUSR0000SAF112",
                "CUSR0000SAF113",
                "CPIFABSL",
                "UNRATE",
                "FEDFUNDS",
                "IRLTLT01CAM156N",
                "LRUNTTTTCAM156S",
                "CPALCY01CAM661N",
                "CPGRLE01CAM657N",
                "QCAR368BIS"
               ]

FRED_API_KEY = "b87bd38f4622b9c19ed583bcfc97bd3e"  # REMOVE ME BEFORE PUBLISHING

expl_df = preprocess_expl(update_expl_data(data_sources, './data_files/expl_vars.csv'), api_key=FRED_API_KEY)
expl_df_monthly = expl_df.resample(frequency).mean().interpolate()
expl_df_monthly

,DEXCAUS,DCOILWTICO,WILL5000IND,VXOCLS,CUSR0000SAF112,CUSR0000SAF113,CPIFABSL,UNRATE,FEDFUNDS,IRLTLT01CAM156N,LRUNTTTTCAM156S,CPALCY01CAM661N,CPGRLE01CAM657N,QCAR368BIS
1986-01-01,1.39,11.13,5.53,24.70,102.50,110.40,107.50,6.70,8.14,10.04,10.00,52.29,0.19,8.76
1986-02-01,1.39,11.13,5.53,24.70,102.00,105.30,107.40,7.20,7.86,9.97,10.00,52.53,0.72,9.85
1986-03-01,1.39,11.13,5.53,24.70,101.90,105.90,107.60,7.20,7.48,9.40,9.90,52.85,0.76,10.95
1986-04-01,1.39,11.13,5.53,24.70,101.10,107.80,107.80,7.10,6.99,8.85,9.80,53.01,0.51,12.04
1986-05-01,1.38,13.80,5.54,21.92,101.20,110.10,108.20,7.20,6.85,8.93,9.70,53.33,0.84,12.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-01,1.22,71.38,211.17,14.94,282.22,316.32,275.90,5.73,0.08,1.36,7.66,110.92,0.30,10.76
2021-07-01,1.25,72.46,216.53,15.37,285.30,315.21,277.39,5.38,0.09,1.25,7.29,111.48,0.22,10.76
2021-08-01,1.26,67.94,220.54,15.08,289.88,315.58,279.34,5.01,0.09,1.23,7.00,111.78,0.18,10.76
2021-09-01,1.27,71.52,221.02,17.49,293.13,316.63,280.61,4.80,0.08,1.26,6.90,111.78,0.18,10.76


## Data Splitting 

In [6]:
report_sim_dates = ["2015-07-01", "2016-07-01", "2017-07-01", "2018-07-01", "2019-07-01", "2020-07-01"]
sim_train_dates = {}
sim_valid_dates = {}

for date in report_sim_dates:
    sim_train_dates[date] = foodprice_df.index[foodprice_df.index <= date]
    sim_valid_dates[date] = foodprice_df.index[(foodprice_df.index > date) & (foodprice_df.index <= (pd.to_datetime(date) + pd.DateOffset(months=18)))]

### Make df for neuralprophet given food_category, dates, and lagged regressor  

In [7]:
def get_neuralprophet_df(foodprice_df, food_category, dates, lagged_regressor):
    prophet_df = pd.DataFrame({'ds': foodprice_df[food_category][dates].index, 
                           'y':foodprice_df[food_category].loc[foodprice_df.index][dates], 
                           lagged_regressor: expl_df_monthly[lagged_regressor].loc[foodprice_df.index][dates],
                          })
    prophet_df.reset_index(drop=True, inplace=True)

    return prophet_df
    


example of train_df:

In [8]:
report_sim_date = "2016-07-01"
report_train_dates = sim_train_dates[report_sim_date]
report_valid_dates = sim_valid_dates[report_sim_date]
train_df = get_neuralprophet_df(foodprice_df, 'Meat', report_train_dates, 'DEXCAUS')
train_df

,ds,y,DEXCAUS
0,1986-01-01,65.1,1.39
1,1986-02-01,64.2,1.39
2,1986-03-01,64.2,1.39
3,1986-04-01,63.6,1.39
4,1986-05-01,64.0,1.38
...,...,...,...
362,2016-03-01,160.0,1.34
363,2016-04-01,158.7,1.30
364,2016-05-01,159.1,1.31
365,2016-06-01,156.7,1.31


#### Define Metrics 

In [9]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

metrics = {
    'r2_score': r2_score,
    'mae': mean_absolute_error,
    'mape': mean_absolute_percentage_error,
    'mse': mean_squared_error,
    'rmse': rmse
}

## Hyperparameter Tuning using ParameterGrid

In [10]:
from sklearn.model_selection import ParameterGrid
params_grid = {'n_forecasts':[18],
              'n_lags':[24, 36, 48, 60]}
grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 4


# Neural Prophet Model Tuning

In [ ]:
%%time
## takes 13 min for one pair of var and p
all_output_paths = []

## we can have these two outside the loop since the key will be based on the output_path and food_category
all_valid_metrics = {}
all_forecasts = {}

for var in data_sources:
    for p in grid:
        details = str(var) + '_nlags_' + str(p['n_lags'])
        output_path = './output/neuralprophet_202110_' + details
        if not os.path.exists(output_path):
            print("making new directory for: ", output_path)
            os.mkdir(output_path)
        all_output_paths.append(output_path)
        
        
        for food_category in foodprice_df.columns:
            
            annual_valid_metrics = {}
            annual_forecasts = {}
                        
            for date in report_sim_dates:
                
                train_dates = sim_train_dates[date]
                valid_dates = sim_valid_dates[date]
                
                
                train_model =NeuralProphet(n_forecasts = p['n_forecasts'], n_lags = p['n_lags'])
                train_df = get_neuralprophet_df(foodprice_df, food_category, train_dates, str(var)) #select data
                train_model = train_model.add_lagged_regressor(names=[str(var)])                   #lagged regressor to model
                train_model.fit(train_df, freq='MS')                                              #fit the model
                
                future = train_model.make_future_dataframe(train_df, periods=18)                #create the future period
                forecast = train_model.predict(future)                                        #produce the future period forecast
                
                all_yhats = []
                for yhat in ["yhat" + str(i) for i in range(1,19)]:
                    all_yhats.append((forecast.loc[forecast[yhat].first_valid_index()][yhat]))  #get all yhats from forecast
                print("this is all yhats ", all_yhats)

                
                valid_df = get_neuralprophet_df(foodprice_df, food_category, valid_dates, str(var))
                
                ##special case for 2020 forecast
                if len(valid_df) == 14:
                    print("case of 2020 forecast")
                    all_yhats = all_yhats[:-4]
                
                ## compute valid metrics
                valid_metrics = {metric_name: metric_fn(y_true=valid_df.y, y_pred=all_yhats) for metric_name, metric_fn in metrics.items()}
                valid_metrics_series = pd.Series(valid_metrics)
                
                annual_valid_metrics[date] = pd.DataFrame(valid_metrics_series).mean(axis=1)
                annual_forecasts[date] = pd.Series(all_yhats)
                            
            
#                 break ### loop for dates 
                
            all_valid_metrics[(output_path, food_category)] = annual_valid_metrics
            all_forecasts[(output_path, food_category)] = annual_forecasts
                
            print("this is all_valid_metrics", all_valid_metrics)
            mean_valid_metrics = {}
            for (output_path,food_category), valid_scores in all_valid_metrics.items():
                mean_valid_metrics[food_category] = pd.DataFrame(valid_scores).mean(axis=1)

            mean_valid_metrics_df = pd.DataFrame(mean_valid_metrics).T
            mean_valid_metrics_df.to_csv(f"{output_path}/mean_fc_valid_metrics.csv")
            
        
#             break ## loop for food category
        
#         break ## loop for n_lags

#     break ##loop for lagged regressors

INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 233


  0%|          | 0/225 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.70E-02, min: 5.62E-01
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 2.77E-01
Epoch[233/233]: 100%|██████████| 233/233 [00:12<00:00, 18.75it/s, SmoothL1Loss=0.000156, MAE=1.07, MSE=2.14, RegLoss=0]
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 231


  0%|          | 0/226 [00:00<?, ?it/s]

this is all yhats  [155.54837036132812, 154.05970764160156, 155.257568359375, 155.21182250976562, 155.75119018554688, 155.86166381835938, 155.98553466796875, 156.18930053710938, 157.82717895507812, 157.71798706054688, 156.5247039794922, 157.3887939453125, 156.78594970703125, 156.2512664794922, 156.91726684570312, 156.76429748535156, 158.18820190429688, 158.54177856445312]


...

this is all yhats  [133.74560546875, 133.65396118164062, 133.08995056152344, 132.778564453125, 131.7306365966797, 133.45742797851562, 133.63890075683594, 134.20742797851562, 134.4766845703125, 134.67857360839844, 134.71632385253906, 135.56558227539062, 136.33685302734375, 136.361083984375, 136.07485961914062, 136.01455688476562, 135.90098571777344, 138.5367431640625]
this is all_valid_metrics {('./output/neuralprophet_202110_DEXCAUS_nlags_24', 'Bakery and cereal products (excluding baby food)'): {'2015-07-01': r2_score    -1.41
mae          2.46
mape         0.02
mse         13.92
rmse         3.73
dtype: float64, '2016-07-01': r2_score    -6.03
mae          3.22
mape         0.02
mse         11.97
rmse         3.46
dtype: float64}, ('./output/neuralprophet_202110_DEXCAUS_nlags_24', 'Dairy products and eggs'): {'2015-07-01': r2_score   -4.58
mae         2.86
mape        0.02
mse         9.40
rmse        3.07
dtype: float64, '2016-07-01': r2_score   -4.78e-01
mae         8.48e-01
mape  

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.58E-03, min: 4.26E-01
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 9.40E-02
Epoch[233/233]: 100%|██████████| 233/233 [00:12<00:00, 18.74it/s, SmoothL1Loss=0.00269, MAE=3.09, MSE=16.5, RegLoss=0]
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 231


  0%|          | 0/226 [00:00<?, ?it/s]

this is all yhats  [120.47831726074219, 112.22647094726562, 111.17327880859375, 119.23192596435547, 119.91389465332031, 123.97791290283203, 125.77377319335938, 125.95155334472656, 126.42929077148438, 126.25956726074219, 130.21829223632812, 127.16065216064453, 121.35968780517578, 114.87736511230469, 117.20398712158203, 124.17236328125, 127.43777465820312, 130.79908752441406]


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.25E-02, min: 6.31E-01
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 2.75E-01
Epoch[206/231]:  89%|████████▉ | 206/231 [00:11<00:01, 17.62it/s, SmoothL1Loss=0.00273, MAE=3.47, MSE=20.2, RegLoss=0] 

In [12]:
print(all_output_paths) 
for output_path, food_category in all_forecasts.keys():
#     print(output_path)
    print(food_category)

['./output/neuralprophet_202110_DEXCAUS_nlags_24']


## Save Validation Forecasts to File (For Ensembling)

In [13]:
date_forecasts = {}
for date in report_sim_dates:
    valid_dates = sim_valid_dates[date]
    food_forecasts = {}
    for my_output_path in all_output_paths:
        for food_category in food_categories:
            food_forecasts[food_category] = all_forecasts[(my_output_path,food_category)][date]
        date_forecasts[date] = pd.DataFrame(food_forecasts)
        date_forecasts[date] = date_forecasts[date].set_index(pd.DatetimeIndex(valid_dates))
        date_forecasts[date].to_csv(f"{my_output_path}/forecasts_{date}.csv")

KeyError: ('./output/neuralprophet_202110_DEXCAUS_nlags_24', 'Bakery and cereal products (excluding baby food)')